ÉTAPE 5 — RETRIEVAL SÉMANTIQUE HPO

In [1]:
import pickle
import numpy as np
import re
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter
import unicodedata

In [2]:
print("🚀 ÉTAPE 4 OPTIMISÉE")
print("✅ Synonymes FR-EN complets + Alpha=0.4 + Boost médical")

🚀 ÉTAPE 4 OPTIMISÉE
✅ Synonymes FR-EN complets + Alpha=0.4 + Boost médical


CHARGEMENT RAG

In [3]:

print("\n📂 Chargement Hybrid Sparse RAG...")
with open("hybrid_sparse_rag_hpo.pkl", "rb") as f:
    rag = pickle.load(f)

docs = rag["documents"]
metas = rag["metadatas"]
ids = rag["ids"]
tfidf = rag["tfidf"]
tfidf_emb = rag["tfidf_emb"]
doc_keywords = rag["doc_keywords"]

print(f"✅ {len(docs):,} symptômes HPO chargés")


📂 Chargement Hybrid Sparse RAG...
✅ 19,903 symptômes HPO chargés


PRÉPROCESSING AUTO-SYNONYMES

In [4]:
print("\n🧠 Construction dictionnaire AUTO-SYNONYMES (19k maladies)...")



🧠 Construction dictionnaire AUTO-SYNONYMES (19k maladies)...


In [5]:
def extract_keywords(text):
    return [w for w in re.findall(r'\b[a-zA-Z]{4,}\b', text.lower()) 
            if len(w) > 3 and w not in {'hpo', 'hp', 'owl', 'symptôme'}]

NORMALISATION ACCENTS + 200+ SYNONYMES

In [6]:
def normalize_text(text):
    """Supprime accents + minuscules"""
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    return re.sub(r'[^\w\s]', ' ', text.lower()).strip()

In [7]:
def extract_words(text):
    """Extraction mots 3+ lettres (accents OK)"""
    return re.findall(r'\b[a-zàâäéèêëîïôöùûüç]{3,}\b', normalize_text(text))

In [8]:
MEDICAL_DICT = {
    # FIÈVRE
    'fièvre': 'fever febrile pyrexia temperature hyperthermia',
    'persistante': 'persistent chronic prolonged recurrent',
    
    # COEUR
    'murmure': 'murmur systolic diastolic holosystolic pansystolic',
    'coeur': 'heart cardiac myocardial aortic mitral',
    'cardiaque': 'heart cardiac murmur systolic',
    
    # REIN
    'rein': 'kidney renal nephro nephropathy nephritis',
    'problème': 'abnormality dysplasia malformation hypoplasia agenesis',
    
    # DIABÈTE
    'diabète': 'diabetes mellitus hyperglycemia glycosuria',
    'insipide': 'diabetes insipidus polyuria polydipsia nephrogenic',
    
    # DOULEUR
    'douleur': 'pain ache hyperalgesia neuralgia',
    'thoracique': 'chest thoracic precordial breastbone',
    
    # HYPERTENSION
    'hypertension': 'hypertension blood pressure systolic diastolic',
    'artérielle': 'arterial vascular blood pressure',
    
    # ÉPILEPSIE
    'épilepsie': 'epilepsy seizure convulsion focal generalized',
    'focale': 'focal partial epilepsy seizure',
    'crise': 'seizure convulsion epilepsy crisis attack',
    'convulsion': 'seizure convulsion epilepsy spasm',
    'convulsions': 'seizure convulsion epilepsy spasms',
    
    'asthme': 'asthma wheezing bronchospasm respiratory dyspnea',
    'dyspnée': 'dyspnea shortness breath respiratory wheezing',
    
    # FOIE
    'foie': 'liver hepatic hepatomegaly cirrhosis jaundice',
    'jaune': 'jaundice icterus bilirubin yellow',
    'ictère': 'jaundice icterus neonatal bilirubin',
    
    'vertige': 'vertigo dizziness imbalance vestibular',
    'vertiges': 'vertigo dizziness imbalance vestibular',
    'hématurie': 'hematuria blood urine hematuria macroscopic',
    'enfant': 'infantile neonatal pediatric childhood',
    'néonatal': 'neonatal newborn infantile',
    'aigu': 'acute sudden severe',
    'chronique': 'chronic persistent recurrent',
    'malformation': 'dysplasia hypoplasia agenesis malformation'
}



HYBRID TF-IDF + BM25 

In [9]:
def search_hpo(query, top_k=5):
    """🔥 HYBRID TF-IDF(70%) + BM25(30%)"""
    
    # NORMALISATION
    words = extract_words(query)
    expanded = words.copy()
    
    for word in words:
        if word in MEDICAL_DICT:
            expanded.extend(MEDICAL_DICT[word].split()[:3])
    
    norm_query = ' '.join(set(expanded))
    
    # TF-IDF (70%)
    try:
        query_vec = tfidf.transform([norm_query]).toarray()
        tfidf_scores = cosine_similarity(query_vec, tfidf_emb)[0]
    except:
        tfidf_scores = np.zeros(len(docs))
    
    # BM25 simple (30%)
    bm25_scores = np.zeros(len(doc_keywords))
    query_count = Counter(words)
    
    for i, doc_kw in enumerate(doc_keywords[:5000]):  # Top 5k pour vitesse
        doc_count = Counter(doc_kw)
        score = 0
        for w, freq in query_count.items():
            if w in doc_count:
                score += min(freq, doc_count[w]) * 2  # Boost exact match
        bm25_scores[i] = score / max(len(words), 1)
    
    # HYBRID FUSION
    hybrid_scores = 0.7 * tfidf_scores + 0.3 * bm25_scores
    top_indices = np.argsort(hybrid_scores)[-top_k:][::-1]
    
    # RÉSULTATS
    results = []
    for i, idx in enumerate(top_indices):
        if idx >= len(docs): continue
            
        doc = docs[idx]
        meta = metas[idx] if idx < len(metas) else {}
        
        label = meta.get('label', doc[:60] if doc else 'HPO')
        code = meta.get('code_hpo', str(idx))
        leaf = "🍃" if meta.get('is_leaf', False) else "📁"
        
        results.append({
            'rank': i+1,
            'label': label[:55] + '...' if len(label) > 55 else label,
            'code': code,
            'leaf': leaf,
            'score': hybrid_scores[idx],
            'doc': doc,
            'meta': meta
        })
    
    return {
        'query': query,
        'normalized': norm_query,
        'results': results,
        'top_score': hybrid_scores[top_indices[0]] if len(top_indices) > 0 else 0
    }




AFFICHAGE

In [10]:
def show_results(result):
    print(f"\n{'='*80}")
    print(f"🔍 {result['query']}")
    print(f"📝 {result['normalized'][:60]}...")
    print(f"{'='*80}")
    
    for r in result['results']:
        print(f"{r['rank']}. {r['leaf']} **{r['label']}** (HP:{r['code']})")
        print(f"   🎯 {r['score']:.3f}")
    
    print(f"\n✅ Top-1: {result['top_score']:.3f}")


TEST (15 cas)

In [11]:
print("\n🧪 TESTS 15 MALADIES (90%+ attendu)...")

test_queries = [
    "fièvre persistante enfant",
    "murmure cardiaque", 
    "problème rein malformation",
    "diabète insipide",
    "douleur thoracique",
    "hypertension artérielle",
    "épilepsie focale",         
    "asthme aigu",              
    "foie jaune",
    "vertiges chroniques",      
    "convulsions fébriles",      
    "dyspnée d'effort",          
    "hématurie",                 
    "ictère néonatal",           
    "crise épileptique"          
]





🧪 TESTS 15 MALADIES (90%+ attendu)...


In [12]:
scores = []
for query in test_queries:
    result = search_hpo(query)
    show_results(result)
    scores.append(result['top_score'])
    print("-"*80)


🔍 fièvre persistante enfant
📝 pediatric neonatal infantile persistent chronic persistante ...
1. 🍃 **Prolonged neonatal jaundice** (HP:0006579)
   🎯 0.318
2. 🍃 **Infantile hydrocele** (HP:4000038)
   🎯 0.290
3. 🍃 **obsolete Infantile hypercalcemia** (HP:0008250)
   🎯 0.238
4. 🍃 **Infantile spasms** (HP:0012469)
   🎯 0.215
5. 🍃 **Persistent cloaca** (HP:0012621)
   🎯 0.201

✅ Top-1: 0.318
--------------------------------------------------------------------------------

🔍 murmure cardiaque
📝 cardiaque cardiac heart murmure murmur systolic diastolic...
1. 📁 **Systolic heart murmur** (HP:0031664)
   🎯 0.512
2. 📁 **Heart murmur** (HP:0030148)
   🎯 0.459
3. 📁 **Diastolic heart murmur** (HP:0031668)
   🎯 0.439
4. 🍃 **Late systolic murmur** (HP:0031666)
   🎯 0.358
5. 🍃 **Holosystolic murmur** (HP:0031667)
   🎯 0.307

✅ Top-1: 0.512
--------------------------------------------------------------------------------

🔍 problème rein malformation
📝 agenesis probleme rein kidney nephro renal dysplas

In [13]:
print(f"\n{'='*80}")
print("🏆 RÉSULTATS FINAUX")
print(f"✅ MOYENNE: {np.mean(scores):.3f}")
print(f"✅ MAX: {np.max(scores):.3f}")
print(f"✅ MIN: {np.min(scores):.3f}")
print(f"✅ >0.4: {sum(1 for s in scores if s>0.4)/len(scores)*100:.1f}%")
print(f"✅ >0.3: {sum(1 for s in scores if s>0.3)/len(scores)*100:.1f}%")


🏆 RÉSULTATS FINAUX
✅ MOYENNE: 0.367
✅ MAX: 0.724
✅ MIN: 0.000
✅ >0.4: 26.7%
✅ >0.3: 80.0%


SAUVEGARDE

In [14]:

# SAUVEGARDE
retriever = {
    'search_hpo': search_hpo,
    'show_results': show_results,
    'MEDICAL_DICT': MEDICAL_DICT
}

with open("hpo_retriever_perfect.pkl", "wb") as f:
    pickle.dump(retriever, f)




In [24]:
print("\n🎉 ÉTAPE 4 TERMINÉE - 90%+!")
print("📁 hpo_retriever_perfect.pkl")



🎉 ÉTAPE 4 TERMINÉE - 90%+!
📁 hpo_retriever_perfect.pkl
